# Greykite example

In [38]:
import pandas as pd
from collections import defaultdict
import warnings
warnings.filterwarnings("ignore")
import plotly
import itertools
import numpy as np
from matplotlib import pyplot as plt
from matplotlib.dates import (
    MonthLocator,
    num2date,
    AutoDateLocator,
    AutoDateFormatter,
)
from matplotlib.ticker import FuncFormatter
from datetime import datetime
from pandas_datareader.data import DataReader

In [15]:
#!pip install greykite

In [16]:
 from greykite.common.data_loader import DataLoader
 from greykite.framework.templates.autogen.forecast_config import ForecastConfig
 from greykite.framework.templates.autogen.forecast_config import MetadataParam
 from greykite.framework.templates.forecaster import Forecaster
 from greykite.framework.templates.model_templates import ModelTemplateEnum
 from greykite.framework.utils.result_summary import summarize_grid_search_results

## Helpers

In [17]:
def subtract_one_month(timestamp):
    import datetime
    return( timestamp - datetime.timedelta(days=30) )

In [18]:
def subtract_one_year(timestamp):
    import datetime
    return( timestamp - datetime.timedelta(days=365) )

## CHF/DKK

In [19]:
item = 'CHFDKK=X'

In [20]:
# Get Data
data = DataReader(item,  "yahoo", datetime(1900,1,1), datetime.now())
data = pd.DataFrame(data)
data.index.name = 'ds'
data.reset_index(inplace=True)
data = data.dropna()
data = data.rename(columns={'Close' : 'y'})

INFO:numexpr.utils:NumExpr defaulting to 8 threads.


In [21]:
data

,ds,High,Low,Open,y,Volume,Adj Close
0,2003-12-01,4.79410,4.773700,4.79160,4.77920,0.0,4.77920
1,2003-12-02,4.78130,4.766700,4.78060,4.77050,0.0,4.77050
2,2003-12-03,4.77280,4.759600,4.77030,4.76550,0.0,4.76550
3,2003-12-04,4.77280,4.759400,4.76510,4.77050,0.0,4.77050
4,2003-12-05,4.78970,4.763900,4.77100,4.78700,0.0,4.78700
...,...,...,...,...,...,...,...
4516,2021-05-27,6.80150,6.777100,6.79700,6.79800,0.0,6.79800
4517,2021-05-30,6.78250,6.743252,6.77630,6.77660,0.0,6.77660
4518,2021-05-31,6.78450,6.741725,6.76670,6.76650,0.0,6.76650
4519,2021-06-01,6.78840,6.746843,6.78410,6.78380,0.0,6.78380


## Greykite Pipeline

In [22]:
 # specify dataset information
 metadata = MetadataParam(
     time_col="ds",  # name of the time column ("date" in example above)
     value_col="y",  # name of the value column ("sessions" in example above)
     freq="D"  # "H" for hourly, "D" for daily, "W" for weekly, etc.
               # Any format accepted by `pandas.date_range`
 )

In [24]:
 forecaster = Forecaster()  # Creates forecasts and stores the result
 result = forecaster.run_forecast_config(  # result is also stored as `forecaster.forecast_result`.
     df=data,
     config=ForecastConfig(
         model_template=ModelTemplateEnum.SILVERKITE.name,
         forecast_horizon=365,  # forecasts 365 steps ahead
         coverage=0.95,         # 95% prediction intervals
         metadata_param=metadata
     )
 )

te-packages/greykite/common/evaluation.py:90: UserWarning:

104 value(s) in y_true were NA or infinite and are omitted in error calc.

/home/oem/anaconda3/lib/python3.8/site-packages/greykite/sklearn/transform/null_transformer.py:177: RuntimeWarning:

Input data has many null values. Missing 29.08% of one input.

/home/oem/anaconda3/lib/python3.8/site-packages/greykite/common/evaluation.py:90: UserWarning:

1541 value(s) in y_true were NA or infinite and are omitted in error calc.

/home/oem/anaconda3/lib/python3.8/site-packages/greykite/sklearn/transform/null_transformer.py:177: RuntimeWarning:

Input data has many null values. Missing 29.08% of one input.

/home/oem/anaconda3/lib/python3.8/site-packages/greykite/common/evaluation.py:90: UserWarning:

1541 value(s) in y_true were NA or infinite and are omitted in error calc.

/home/oem/anaconda3/lib/python3.8/site-packages/greykite/sklearn/transform/null_transformer.py:177: RuntimeWarning:

Input data has many null values. Missing 29.

## Cross-validation

In [29]:
 grid_search = result.grid_search
 cv_results = summarize_grid_search_results(
     grid_search=grid_search,
     decimals=2,
     # The below saves space in the printed output. Remove to show all available metrics and columns.
     cv_report_metrics=None,
     column_order=["rank", "mean_test", "split_test", "mean_train", "split_train", "mean_fit_time", "mean_score_time", "params"])
 # Transposes to save space in the printed output
 cv_results["params"] = cv_results["params"].astype(str)
 cv_results.set_index("params", drop=True, inplace=True)
 cv_results.transpose()

params,[]
rank_test_MAPE,1
mean_test_MAPE,53.3
split_test_MAPE,"(3.31, 92.61, 63.98)"
mean_train_MAPE,43.64
split_train_MAPE,"(0.58, 66.51, 63.81)"
mean_fit_time,17.41
mean_score_time,0.65


### Backtest

In [35]:
 backtest = result.backtest
 fig = backtest.plot()

ModuleNotFoundError: No module named 'plotly.validator_cache'

### Metrics

In [39]:
 backtest_eval = defaultdict(list)
 for metric, value in backtest.train_evaluation.items():
     backtest_eval[metric].append(value)
     backtest_eval[metric].append(backtest.test_evaluation[metric])
 metrics = pd.DataFrame(backtest_eval, index=["train", "test"]).T
 metrics

,train,test
CORR,0.0943161,0.0197351
R2,0.00837261,-6093.54
MSE,1540.71,41.3167
RMSE,39.2519,6.42781
MAE,5.8966,4.05514
MedAE,2.79471,1.3303
MAPE,60.4476,59.0632
MedAPE,58.1005,19.3426
sMAPE,21.0172,18.0092
Q80,2.73882,1.01777


In [42]:
result.model[-1].summary()

================================ Model Summary =================================

Number of observations: 6395,   Number of features: 122
Method: Ridge regression
Number of nonzero features: 122
Regularization parameter: 1.000e+05

Residuals:
         Min           1Q       Median           3Q          Max
      -11.43        -4.93       -3.152       -1.002        606.8

             Pred_col    Estimate   Std. Err Pr(>)_boot sig. code                     95%CI
            Intercept       8.135     0.4967     <2e-16       ***            (7.199, 9.115)
  events_C...New Year    0.003312   0.003909      0.652               (-0.0009369, 0.01136)
  events_C...w Year-1    0.001382   0.001894      0.620              (-0.0008738, 0.005447)
  events_C...w Year-2  -0.0004814  0.0001639      0.004        **    (-0.00085, -0.0002245)
  events_C...w Year+1    0.005263   0.006201      0.602               (-0.0009718, 0.02275)
  events_C...w Year+2  -0.0008054  0.0002687      0.008        **   (-0.00

In [41]:
result.backtest.df

,ds,actual,forecast,forecast_lower,forecast_upper
0,2003-12-01,4.77920,8.232051,-142.043604,158.507705
1,2003-12-02,4.77050,8.016040,5.996086,10.035993
2,2003-12-03,4.76550,8.110608,5.012914,11.208303
3,2003-12-04,4.77050,8.053784,5.685203,10.422365
4,2003-12-05,4.78700,8.093017,-19.155709,35.341742
...,...,...,...,...,...
6390,2021-05-30,6.77660,20.061569,-108.534186,148.657323
6391,2021-05-31,6.76650,17.403677,-132.871977,167.679331
6392,2021-06-01,6.78380,7.124467,5.104514,9.144421
6393,2021-06-02,NaN,5.098391,2.000697,8.196085


In [40]:
result.forecast.df

,ds,actual,forecast,forecast_lower,forecast_upper
0,2003-12-01,4.7792,8.434334,-137.615114,154.483782
1,2003-12-02,4.7705,8.221028,5.927322,10.514735
2,2003-12-03,4.7655,8.310295,5.077554,11.543035
3,2003-12-04,4.7705,8.257330,5.654325,10.860335
4,2003-12-05,4.7870,8.295371,-18.182483,34.773225
...,...,...,...,...,...
6755,2022-05-30,NaN,16.508481,-129.540967,162.557929
6756,2022-05-31,NaN,6.840650,4.546944,9.134357
6757,2022-06-01,NaN,5.120081,1.887340,8.352822
6758,2022-06-02,NaN,6.274590,3.671585,8.877595


In [ ]:
def collect_tune_and_predict(item,fred,n_ahead):
    param_grid =    {  
            'changepoint_prior_scale': [0.01, 0.1, 1.0, 5.0, 10.0],
            'seasonality_prior_scale': [0.01, 0.1, 1.0, 5.0, 10.0,],
                        }
    # Get Data
    data = fred.get_series_latest_release(item)
    data = pd.DataFrame(data)
    data.columns = ["y"]
    data['ds'] = data.index
    data = data.dropna()

    # Tune
    all_params = [dict(zip(param_grid.keys(), v)) for v in itertools.product(*param_grid.values())]
    rmses = [] 
    # Use cross validation to evaluate all parameters
    latest_date = data.tail(1).iloc[0]['ds']
    cutoffs = [ subtract_one_month( latest_date ) ]
    for params in all_params:
        m = Prophet(**params).fit(data)  # Fit model with given params
        df_cv = cross_validation(m, cutoffs=cutoffs, horizon='30 days', parallel="processes")
        df_p = performance_metrics(df_cv, rolling_window=1)
        rmses.append(df_p['rmse'].values[0])

    # Rolling windows
    # Find the best parameters
    tuning_results = pd.DataFrame(all_params)
    tuning_results['rmse'] = rmses
    # Python
    best_params = all_params[np.argmin(rmses)]

    # Python
    m = Prophet(**best_params)
    m.fit(data)
    
    best_params['rmse'] = min( rmses )
    print(best_params)
    # Python
    future = m.make_future_dataframe(periods=n_ahead)
    forecast = m.predict(future)
    # SMA
    sma = pd.DataFrame(data = { 
                                'ds' : data['ds'], 
                                'SMA_50' : data['y'].rolling(window=50,min_periods=1).mean(), 
                                'SMA_100' : data['y'].rolling(window=100,min_periods=1).mean(),
                                'SMA_200' : data['y'].rolling(window=200,min_periods=1).mean()
    })
    # Join SMA
    forecast = pd.merge(forecast,sma,on='ds',how='left')
    fig1 = plot_(m,forecast)
    a = add_changepoints_to_plot(fig1.gca(), m, forecast)
    # Python
    fig2 = m.plot_components(forecast)

    # Latest 365
    future = m.make_future_dataframe(periods=30)
    forecast = m.predict(future)
    forecast = pd.merge(forecast,sma,on='ds',how='left')
    forecast = forecast[forecast['ds'] >= subtract_one_year( datetime.today() )]
    m.history = m.history[m.history['ds'] >= subtract_one_year( datetime.today() )]
    fig3 = plot_(m,forecast)
    return( { 'fig1' : fig1, 'fig2' : fig2, 'fig3': fig3})

In [ ]:
def plot_(
    m, fcst, ax=None, uncertainty=True, plot_cap=True, xlabel='ds', ylabel='y',
    figsize=(10, 6), include_legend=False
):
    """Plot the Prophet forecast.
    Parameters
    ----------
    m: Prophet model.
    fcst: pd.DataFrame output of m.predict.
    ax: Optional matplotlib axes on which to plot.
    uncertainty: Optional boolean to plot uncertainty intervals, which will
        only be done if m.uncertainty_samples > 0.
    plot_cap: Optional boolean indicating if the capacity should be shown
        in the figure, if available.
    xlabel: Optional label name on X-axis
    ylabel: Optional label name on Y-axis
    figsize: Optional tuple width, height in inches.
    include_legend: Optional boolean to add legend to the plot.
    Returns
    -------
    A matplotlib figure.
    """
    if ax is None:
        fig = plt.figure(facecolor='w', figsize=figsize)
        ax = fig.add_subplot(111)
    else:
        fig = ax.get_figure()
    fcst_t = fcst['ds'].dt.to_pydatetime()
    ax.plot(m.history['ds'].dt.to_pydatetime(), m.history['y'], 'k.',
            label='Observed data points')
    ax.plot(fcst_t, fcst['yhat'], ls='-', c='#0072B2', label='Forecast')
    if 'cap' in fcst and plot_cap:
        ax.plot(fcst_t, fcst['cap'], ls='--', c='k', label='Maximum capacity')
    if m.logistic_floor and 'floor' in fcst and plot_cap:
        ax.plot(fcst_t, fcst['floor'], ls='--', c='k', label='Minimum capacity')
    if uncertainty and m.uncertainty_samples:
        ax.fill_between(fcst_t, fcst['yhat_lower'], fcst['yhat_upper'],
                        color='#0072B2', alpha=0.2, label='Uncertainty interval')
    # Specify formatting to workaround matplotlib issue #12925
    locator = AutoDateLocator(interval_multiples=False)
    formatter = AutoDateFormatter(locator)
    ax.xaxis.set_major_locator(locator)
    ax.xaxis.set_major_formatter(formatter)
    ax.grid(True, which='major', c='gray', ls='-', lw=1, alpha=0.2)
    ax.set_xlabel(xlabel)
    ax.set_ylabel(ylabel)
    # SMA
    ax.plot(fcst['ds'],fcst['SMA_50'] )
    ax.plot(fcst['ds'],fcst['SMA_100'] )
    ax.plot(fcst['ds'],fcst['SMA_200'] )
    if include_legend:
        ax.legend()
    fig.tight_layout()
    return fig